In [ ]:
gpu_info = !nvidia-smi
gpu_info='\n'.join(gpu_info)
if gpu_info.find('failed') >=0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!pip install -q transformers datasets diffusers soundfile

In [ ]:
#Imports

import torch
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio

In [ ]:
#Sentiment Analysis

classifier =pipeline("sentiment-analysis", device="cuda")
result = classifier("I'm super excited to be on the way to LLM mastery from udemy!")
print(result)

In [ ]:
#Named Entity Reconginition

ner = pipeline("ner",grouped_entities=True, device="cuda")
result = ner("Barack  Obama was the 44th president of the United States.")
print(result)

In [ ]:
# Question Answering with Context

question_answerer = pipeline("question-answering",device="cuda")
result = question_answerer(question="who was the 44th president of United States",context="Barack  Obama was the 44th president of the United States.")
print(result)

In [ ]:
from transformers import pipeline

# Initialize the summarization pipeline with GPU acceleration
summarizer = pipeline("summarization", device=0)  # device=0 for GPU (CUDA), -1 for CPU

# Input text for summarization
text = """I had a productive and insightful discussion with my appraiser. I am satisfied with the feedback given and appreciate the time my manager spent sharing comprehensive insights about my strengths and areas for improvement. The feedback was clear, well-structured, and helped me understand my performance and growth opportunities.
I appreciate my manager's commitment to fostering an inclusive work environment and their ability to address and resolve any concerns related to inclusion. Their support and guidance make a positive impact on my professional development."""

# Generate the summary
summary = summarizer(text, max_length=50, min_length=25, do_sample=False)

# Print the summarized text
print(summary[0]['summary_text'])


In [ ]:
# Translation

translator = pipeline("translation_en_to_fr",device="cuda")
result = translator("The Data Scientists were truely amazed by the power and simplicity of the HuggingFace pipeline API.")
print(result[0]['translation_text'])

In [ ]:
# Classification

classifier = pipeline("zero-shot-classification", device="cuda")
result = classifier("Hugging Face's Transformers library is amazing!",candidate_labels = ["technology","sports","politics"])
print(result)

In [ ]:
# Text Generation

generator = pipeline("text-generation",device="cuda")
result = generator("if there's one thing I want to remember about using HiggingFace pipelines, it's")
print(result[0]['generated_text'])

In [ ]:
# Image Generation

image_gen = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16,
    use_safetensors=False,
    variant="fp16"
).to("cuda")

text = "A class of data scientists learning about AI, in the surreal style of money heist"
image = image_gen(text).images[0]
image

In [ ]:
# Image Generation Using Flux Model

image_gen = DiffusionPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.float16
).to("cuda")

text = "A class of data scientists learning about AI, in the surreal style of money heist"
image = image_gen(text).images[0]
image

In [ ]:
# Audio Generation

synthesiser = pipeline("text-to-speech","microsoft/speecht5_tts", device="cuda")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = synthesiser("Hi to an artificial intelligence negineer, on the way to mastery!", forward_params={"speaker_embeddings":speaker_embedding})
sf.write("speech.wav",speech["audio"], samplerate=speech["sampling_rate"])
Audio("speech.wav")